In [ ]:
#%pip install pyarrow
#%pip install -q google-generativeai
#%pip install python-dotenv
#%pip install plotly
#%pip install nbformat --upgrade

In [6]:
import requests
import pandas as pd
from dotenv import load_dotenv
import os
import logging

load_dotenv() 

API_KEY = os.getenv("API_KEY")
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")


In [ ]:
import logging #Util para criar logs de cordo com as açoes especificas que você definir

# Criar um objeto logger com o nome 'api_logger'
api_logger = logging.getLogger('api_logger')

# Definir o nível de registro como DEBUG. Isso significa que todas as mensagens com nível DEBUG e acima serão registradas
api_logger.setLevel(logging.DEBUG)

# Criar manipuladores que imprimem mensagens de log no console (saída padrão) e escrevem mensagens de log em um arquivo chamado 'api_logger.log'
console_handler = logging.StreamHandler()
file_handler = logging.FileHandler('api_logger.log')

# Definir um formatador que especifica o formato das mensagens de log
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# Definir o formatador para o manipulador do console e para o manipulador de arquivos
console_handler.setFormatter(formatter)
file_handler.setFormatter(formatter)

# Adicionar o manipulador do console ao logger. Isso significa que mensagens de log serão exibidas no console
api_logger.addHandler(console_handler)
# Adicionar o manipulador de arquivos ao logger. Isso significa que mensagens de log serão escritas no arquivo 'api_logger.log'
api_logger.addHandler(file_handler)

In [18]:

BASE_URL = f'https://v6.exchangerate-api.com/v6/{API_KEY}'
DATA = '2025/08/25' # Substitua pela data que você quiser

URL = f'{BASE_URL}/latest/USD'

try:
    response = requests.get(URL)
    response.raise_for_status()  # Isso vai levantar um erro para códigos de status HTTP 4xx/5xx

    data = response.json()
    #print(data)

except requests.exceptions.HTTPError as err:
    print(f"Ocorreu um erro HTTP: {err}")
except Exception as err:
    print(f"Ocorreu um erro: {err}")

api_logger.info("Requisição bem-sucedida.")    

2025-09-10 21:39:33,400 - api_logger - INFO - Requisição bem-sucedida.


In [ ]:
# Salvar o retorno como arquivo JSON na pasta bronze, particionando por timestamp
import os
from datetime import datetime

timestamp = datetime.now().strftime("%Y%m%d")
output_dir = f"bronze/{timestamp}"
os.makedirs(output_dir, exist_ok=True)

with open(f"{output_dir}/retorno_api_{timestamp}.json", "w", encoding="utf-8") as f:
    import json
    json.dump(data, f, ensure_ascii=False, indent=4)

api_logger.info(f"Arquivo .json salvo na camada /bronze particionado por timestamp {timestamp}.")    

2025-09-10 21:45:04,551 - api_logger - INFO - Arquivo .json salvo na camada bronze particionado por timestamp 20250910.


In [20]:
# Ler o JSON salvo e salvar como Parquet na pasta silver
import pandas as pd
output_dir = f"bronze/{timestamp}"

with open(f"{output_dir}/retorno_api_{timestamp}.json", "r", encoding="utf-8") as f:
    json_data = json.load(f)

# Normaliza o JSON para DataFrame
df = pd.json_normalize(json_data['conversion_rates'])

silver_dir = f"silver/{timestamp}"
os.makedirs(silver_dir, exist_ok=True)
df.to_parquet(f"{silver_dir}/retorno_api_{timestamp}.parquet", index=False)

api_logger.info(f"Arquivo .parquet salvo na camada /silver particionado por timestamp {timestamp}.")  

2025-09-10 21:45:43,710 - api_logger - INFO - Arquivo .parquet salvo na camada /silver particionado por timestamp 20250910.


In [21]:
#print(f"O valor do dólar sobre o euro é: {df['EUR'].values[0]}")

# Carregar o DataFrame diretamente do arquivo Parquet salvo na pasta silver
silver_dir = f"silver/{timestamp}"
df = pd.read_parquet(f"{silver_dir}/retorno_api_{timestamp}.parquet")

# Filtrar e mostrar apenas as moedas da América do Sul
south_american_currencies = ['USD', 'ARS', 'BOB', 'BRL', 'CLP', 'COP', 'PEN', 'UYU', 'VES', 'GYD', 'PYG', 'SRD']
df_south_america = df[south_american_currencies]
# ...restante do código...

#salvando o DataFrame filtrado como parquet na pasta gold

gold_dir = f"gold/{timestamp}"
os.makedirs(gold_dir, exist_ok=True)
df_south_america.to_parquet(f"{gold_dir}/south_american_currencies.parquet", index=False)   

api_logger.info(f"Criação da lista de moedas sul americanas: {south_american_currencies}.")  

2025-09-10 21:47:12,104 - api_logger - INFO - Criação da lista de moedas sul americanas: ['USD', 'ARS', 'BOB', 'BRL', 'CLP', 'COP', 'PEN', 'UYU', 'VES', 'GYD', 'PYG', 'SRD'].


In [22]:
#Consollidano os resultado para um dataframe final 

gold_base = "gold"
consolidated = []

for folder in sorted(os.listdir(gold_base)):
    folder_path = os.path.join(gold_base, folder)
    if os.path.isdir(folder_path):
        for file in os.listdir(folder_path):
            if file.endswith(".parquet"):
                df = pd.read_parquet(os.path.join(folder_path, file))
                # Se não existe coluna USD, adiciona com valor 1
                if "USD" not in df.columns:
                    df["USD"] = 1
                # Adiciona coluna com o nome da pasta (timestamp)
                df["timestamp"] = folder
                consolidated.append(df)

# Junta todos os DataFrames
df_consolidated = pd.concat(consolidated, ignore_index=True)

# Reorganiza para ter timestamp como linha e moedas como colunas
df_final = df_consolidated.set_index("timestamp")

# Exibe o DataFrame consolidado
api_logger.info(f"Exibindo dataframe com os resultados de cada partição da camada /gold.")  
df_final

2025-09-10 21:48:02,507 - api_logger - INFO - Exibindo dataframe com os resultados de cada partição da camada /gold.


,ARS,BOB,BRL,CLP,COP,PEN,UYU,VES,GYD,PYG,SRD,USD
timestamp,,,,,,,,,,,,
20250904,1360.17,6.9203,5.4534,973.4898,4004.8407,3.5341,40.0037,151.7627,209.0719,7228.9684,38.6265,1
20250909,1422.17,6.9105,5.4139,965.7960,3964.2089,3.5160,40.0851,154.9825,209.1806,7218.2281,38.9228,1
20250910,1419.92,6.9303,5.4311,967.9827,3918.2402,3.4911,39.9888,157.7287,208.9181,7156.3980,38.9628,1


In [23]:
import plotly.graph_objects as go
import plotly.io as pio

# df_final: DataFrame consolidado, index=timestamp, columns=moedas
df_final = df_final.sort_index()  # Garante ordem cronológica

# Lista de moedas disponíveis (exceto USD)
moedas = [col for col in df_final.columns if col != "USD"]

# Cria o gráfico sem o USD
fig = go.Figure()

# Adiciona as moedas como traces, inicialmente invisíveis
for moeda in moedas:
    fig.add_trace(go.Scatter(
        x=df_final.index,
        y=df_final[moeda],
        mode="lines+markers",
        name=moeda,
        visible=False
    ))

# Botões para selecionar moedas
buttons = []
for i, moeda in enumerate(moedas):
    visible = [j == i for j in range(len(moedas))]
    buttons.append(dict(
        label=moeda,
        method="update",
        args=[{"visible": visible},
              {"title": f"Variação da moeda {moeda}"}]
    ))

fig.update_layout(
    updatemenus=[dict(
        type="dropdown",
        x=1.15,
        y=1,
        showactive=True,
        buttons=buttons
    )],
    title="Variação das moedas da América do Sul",
    xaxis_title="Timestamp",
    yaxis_title="Valor em relação ao USD",
    legend_title="Moeda",
    height=600
)

fig.show()

In [24]:
#Conecta Gemini 

import google.generativeai as genai

genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

model = genai.GenerativeModel("gemini-1.5-flash")

def gerar_insights_gemini(df, n_linhas=10):
    """
    Gera insights usando Google Gemini Free API
    :param df: DataFrame a ser analisado
    :param n_linhas: Número de linhas do dataset para enviar ao modelo
    """
    sample = df.head(n_linhas).to_string()
    prompt = f"""
    Você é um analista de dados. Analise o seguinte dataset de moedas da América do Sul em relação ao dólar:

    {sample}

    Gere insights claros e objetivos:
    - Quais moedas estão mais valorizadas e mais desvalorizadas?
    - Quais padrões interessantes podem ser observados?
    - Quais análises adicionais poderiam ser feitas com este dataset?
    - O dólar comparado com o valor do Real está alto. O que pode influenciar esse valor? Isso acompanha os valores das outras moedas da america do sul?
    - Você pode pontuar eventos mais recentes que contribuem para a desvalorização do real em relação ao Dolar? Considerando que a data da cotação como {timestamp }

    """

    response = model.generate_content(prompt)
    return response.text

# Exemplo de uso:
insights = gerar_insights_gemini(df_south_america)
print("=== Insights gerados pelo Gemini ===")
api_logger.info(f"Exibindo retorno dos insights do LLM Google gemini.")  
print(insights)


2025-09-10 21:49:47,964 - api_logger - INFO - Exibindo retorno dos insights do LLM Google gemini.


=== Insights gerados pelo Gemini ===
Análise do Dataset de Moedas da América do Sul em Relação ao Dólar (2025-09-10 - Data Assumida)

O dataset apresentado mostra a taxa de câmbio de várias moedas sul-americanas em relação ao dólar americano (USD) em um único ponto no tempo (assumindo 2025-09-10, já que a data não foi especificada).  A análise se limita a essa única observação, impossibilitando a identificação de tendências ao longo do tempo.

**Insights:**

* **Moedas mais e menos valorizadas:**  Considerando 1 USD como referência, a moeda mais valorizada é o PEN (Sol Peruano) com aproximadamente 3.49 SOL por 1 USD, enquanto a menos valorizada é o VES (Bolívar Venezuelano) com 157.73 VES por 1 USD.  O ARS (Peso Argentino) também apresenta uma taxa de câmbio muito desvalorizada.

* **Padrões interessantes:** A observação de um único dia dificulta a identificação de padrões. Para observar padrões, uma série temporal de dados seria necessária.  No entanto, a grande diferença entre as tax